In [18]:
from node2vec_embeddings import Node2VecEmbeddings

gen = Node2VecEmbeddings(dimensions=128, walk_length=30, num_walks=200, workers=4)

graph = gen.load_turtle_graph("data/ml-100k/graph.turtle")

Loading RDF graph...


In [10]:
# Verify that the graph is loaded correctly
edge_count = 0
for triple in graph:
    edge_count += 1

print("Nodes: ", len(graph.all_nodes()))
print("Edges: ", edge_count)

Nodes:  155603
Edges:  513452


In [17]:
# Encode nodes and edges
gen.encode_graph()

# Create edge DataFrame


AttributeError: 'Node2VecEmbeddings' object has no attribute 'encode_graph'